In [15]:
%reload_ext autoreload 
%autoreload 2

import pandas as pd
import numpy as np
import re
import os
import sys
import time
sys.path.append("../../")
from typing import List, Dict
from os.path import join, dirname
from pydrive.auth import GoogleAuth
from datetime import datetime
from utils import gdrive_downloader

In [2]:
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '../../utils/client_secrets.json'
drive = gdrive_downloader.gdrive_authenticate()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=17797463459-h3beca6cno86q9eu3of312ki2g8dv7f5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [13]:
df_dict = gdrive_downloader.download_data(drive)
[{key: len(value)} for key, value in df_dict.items()]

pageviews
0.0
events
0.0
commerce
0.0


[{'pageviews': 254462}, {'events': 14348}, {'commerce': 1043}]

In [14]:
df_dict['pageviews'].head()

,name,time,_article,action,article_id,category,host,referer,remp_session_id,signed_in,...,tags,title_variant,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,pageviews,1517616000077000000,1,load,6734.0,nezaradene,npress-influxdb-n2.ewhs.bntb.net,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,...,NaN,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/6734/biznis-s-titulmi-kto-p...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
1,pageviews,1517616000078000000,1,load,165977.0,veda,npress-influxdb-n2.ewhs.bntb.net,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,...,NaN,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/165977/zivocisne-druhy-maso...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
2,pageviews,1517616000078000000,1,load,594122.0,komentare,npress-influxdb-n2.ewhs.bntb.net,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,...,NaN,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/594122/mohamedov-zivot/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
3,pageviews,1517616000078000000,1,load,808523.0,hlavna,npress-influxdb-n2.ewhs.bntb.net,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,...,NaN,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/minuta/808523/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
4,pageviews,1517616000078000000,1,load,859254.0,hlavna,npress-influxdb-n2.ewhs.bntb.net,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,...,kvalifikacia-ms-2018,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/minuta/859254/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN


Reshaping data types

In [406]:
df_dict['pageviews'].rename(columns={'_article': 'is_article'}, inplace=True)
df_dict['pageviews']['article_id'].fillna('', inplace=True)
df_dict['pageviews']['is_article'] = df_dict['pageviews']['is_article'].astype(bool)
df_dict['pageviews']['article_id'] = df_dict['pageviews']['article_id'].astype(str)
df_dict['pageviews']['time'] = df_dict['pageviews']['time'].astype(datetime)
df_dict['pageviews']['remp_session_id'].fillna('', inplace = True)
df_dict['pageviews']['signed_in'].fillna('', inplace = True)
df_dict['pageviews']['social'].fillna('', inplace = True)
df_dict['pageviews']['tags'].fillna('', inplace = True)
df_dict['pageviews']['title'].fillna('', inplace = True)
df_dict['pageviews']['user_agent'].fillna('', inplace = True)
[df_dict['pageviews'][column].fillna('', inplace = True) for column in df_dict['pageviews'].columns if re.search('utm_', column) is not None]
df_dict['pageviews'].head()

,is_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,False,load,,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,,,,,1515801600076000000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/autor/bella/,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
1,True,load,294893.0,slovensko,npress-influxdb-n2.ewhs.bntb.net,pageviews,,,,vybuchy-v-parizi,1515801600078000000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/294893/fico-bezpecnost-u-na...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
2,True,load,544744.0,slovensko,npress-influxdb-n2.ewhs.bntb.net,pageviews,,,,,1515801600078000000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/544744/ciznar-a-matovic-si-...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
3,True,load,704078.0,komentare,npress-influxdb-n2.ewhs.bntb.net,pageviews,,,,,1515801600078000000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/704078/za-boha-a-za-narod-b...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
4,True,load,953687.0,nezaradene,npress-influxdb-n2.ewhs.bntb.net,pageviews,,,,rozhovory,1515801600078000000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/953687/ako-evanjelici-sami-...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,


In [407]:
# This function assumes all missing values are either NaN or ''
def get_missing_values_share(table_name: str) -> pd.DataFrame:
    missing_values_share = pd.Series(index=df_dict[table_name].columns)
    num_df_rows = len(df_dict[table_name])
    for column in df_dict[table_name].columns:
        non_na_rows = df_dict[table_name][column].dropna()
        if str(non_na_rows.dtype) == 'object':
            num_relevant_values = len(non_na_rows[non_na_rows != ''])
        else:
            num_relevant_values = len(non_na_rows)
        missing_values_share[column] = num_relevant_values / num_df_rows
    
    return missing_values_share

In [408]:
get_missing_values_share('pageviews')

is_article         1.000000
action             1.000000
article_id         0.525908
category           0.999994
host               1.000000
name               1.000000
remp_session_id    0.011186
signed_in          0.011230
social             0.104669
tags               0.159106
time               1.000000
title              0.000005
token              1.000000
url                1.000000
user_agent         0.000002
user_id            1.000000
utm_campaign       0.012994
utm_content        0.010596
utm_medium         0.013796
utm_source         0.013797
dtype: float64

Look at signed_in values

In [338]:
df_dict['pageviews'][df_dict['pageviews']['signed_in'] == 1].head(3)

,is_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
796,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,74014bf5-abc9-4d25-8aec-baea54ab0b61,1.0,,,2018-01-17 00:11:32.094000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/,,122004,,,,
983,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,74014bf5-abc9-4d25-8aec-baea54ab0b61,1.0,,,2018-01-17 00:14:51.644000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/,,122004,,,,
4230,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,d8e82e4b-9a7e-4505-9614-3cc9a4350244,1.0,,,2018-01-17 01:42:16.587000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/,,39848,,,,


In [339]:
df_dict['pageviews'][df_dict['pageviews']['signed_in'] == 0].head(3)

,is_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,,0.0,,,2018-01-17 00:00:00.076000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/veda/page/5/,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
3,True,load,253261.0,zdravie,npress-influxdb-n2.ewhs.bntb.net,pageviews,,0.0,,,2018-01-17 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/253261/opice-sa-vekom-zhors...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
4,True,load,269636.0,ekonomika,npress-influxdb-n2.ewhs.bntb.net,pageviews,,0.0,,,2018-01-17 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/269636/platia-na-poste-seko...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,


Tags by whether the url is an article

In [350]:
df_dict['pageviews'][df_dict['pageviews']['is_article'] == True]['tags'].value_counts().head()

                          3387809
rozhovory                  230977
ceske-volby-prezidenta     121159
hoaxy-a-propaganda          91298
slovenske-myty              75090
Name: tags, dtype: int64

In [397]:
unique_articles = df_dict['pageviews'].loc[
    df_dict['pageviews']['is_article'] == True, 
    ['article_id', 'tags']].drop_duplicates()
print('share articles without tags', 
      len(unique_articles[unique_articles['tags'] == '']) / len(unique_articles))

share articles without tags 0.7579430670339761


In [349]:
df_dict['pageviews'][df_dict['pageviews']['is_article'] != True]['tags'].value_counts()

    4378740
Name: tags, dtype: int64

Look at article_id by is_article

In [380]:
df_dict['pageviews'].loc[df_dict['pageviews']['article_id'] != '', 'is_article'].value_counts()

True    4857323
Name: is_article, dtype: int64

In [381]:
df_dict['pageviews'].loc[df_dict['pageviews']['article_id'] == '', 'is_article'].value_counts()

False    4378740
Name: is_article, dtype: int64

In [402]:
print(df_dict['events'].columns)
df_dict['events'].head()

Index(['action', 'banner_id', 'campaign_id', 'category', 'email', 'host',
       'mail_job_batch_id', 'name', 'new_email', 'original_email',
       'remp_session_id', 'signed_in', 'social', 'template_code', 'time',
       'token', 'url', 'user_agent', 'user_id', 'utm_campaign', 'utm_content',
       'utm_medium', 'utm_source'],
      dtype='object')


,action,banner_id,campaign_id,category,email,host,mail_job_batch_id,name,new_email,original_email,...,template_code,time,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,show,NaN,NaN,banner,NaN,npress-influxdb-n2.ewhs.bntb.net,NaN,events_v2,NaN,NaN,...,NaN,1515801600121000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/353377/skriniar-odchadza-zo...,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,16fd6c84-d6c9-4474-b90c-a548f8ab39ba,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
1,show,NaN,NaN,banner,NaN,npress-influxdb-n2.ewhs.bntb.net,NaN,events_v2,NaN,NaN,...,NaN,1515801600130000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/389971/chronicky-stres-uryc...,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,16fd6c84-d6c9-4474-b90c-a548f8ab39ba,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
2,show,NaN,NaN,banner,NaN,npress-influxdb-n2.ewhs.bntb.net,NaN,events_v2,NaN,NaN,...,NaN,1515801600136000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/autor/bella/,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,16fd6c84-d6c9-4474-b90c-a548f8ab39ba,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
3,show,NaN,NaN,banner,NaN,npress-influxdb-n2.ewhs.bntb.net,NaN,events_v2,NaN,NaN,...,NaN,1515801600138000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/minuta/997484/,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,16fd6c84-d6c9-4474-b90c-a548f8ab39ba,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
4,show,NaN,NaN,banner,NaN,npress-influxdb-n2.ewhs.bntb.net,NaN,events_v2,NaN,NaN,...,NaN,1515801600139000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/minuta/969399/,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,16fd6c84-d6c9-4474-b90c-a548f8ab39ba,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign


In [409]:
df_dict['events']['time'] = df_dict['events']['time'].astype(datetime)
df_dict['events'].fillna('', inplace = True)

In [427]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
events_missing_values_share = get_missing_values_share('events')
events_missing_values_share

action              1.00000
banner_id           0.00000
campaign_id         0.00000
category            1.00000
email               0.41400
host                1.00000
mail_job_batch_id   0.41400
name                1.00000
new_email           0.00001
original_email      0.00001
remp_session_id     0.04879
signed_in           0.04891
social              0.10815
template_code       0.41400
time                1.00000
token               1.00000
url                 0.58584
user_agent          0.00000
user_id             1.00000
utm_campaign        0.58459
utm_content         0.58459
utm_medium          0.58459
utm_source          0.58459
dtype: float64

In [414]:
df_dict['events'][df_dict['events']['signed_in'] == 1].head(3)['remp_session_id']

25    d7cf7eef-922f-48d5-9773-f36896a7e466
26    19267cd2-58f3-4419-80bc-ccd69800edfe
31    8814d446-7d13-455a-8278-638f7c3c276c
Name: remp_session_id, dtype: object

In [419]:
df_dict['events'].loc[df_dict['events']['signed_in'] == 0, 'remp_session_id'].value_counts().head()

                                        218
20440db4-2cde-4aab-abce-369b64ba6def     18
d54453b4-5242-4d7a-86ce-791ffa3b3b36     16
5242d7a0-6ce7-491f-ba3b-3b36d94d0cd4      7
53b45242-d7a0-46ce-b91f-fa3b3b36d94d      7
Name: remp_session_id, dtype: int64

In [422]:
df_dict['events']['url'].value_counts().head()

                                                                                                                       1080714
https://dennikn.sk/                                                                                                     849935
https://dennikn.sk/?ref=menu                                                                                             11456
https://dennikn.sk/980405/nie-sme-vhodni-pre-deti-upozornuje-metalova-kaviaren-v-bratislave-po-zlych-skusenostiach/       9060
https://dennikn.sk/977427/vobec-nepotrebujete-predajnu-tiger-rozmyslite-si-ci-tam-nakupujete/                             8590
Name: url, dtype: int64

In [424]:
df_dict['events'].loc[df_dict['events']['url'] == '', 'email'].value_counts().head()

                                412
radoslav.augustin@dennikn.sk    111
a@avenarius.sk                  100
talarovic1@gmail.com             99
chovan@agropartner.sk            99
Name: email, dtype: int64

In [425]:
df_dict['events'].loc[df_dict['events']['email'] == '', 'url'].value_counts().head()

https://dennikn.sk/                                                                                                    849935
https://dennikn.sk/?ref=menu                                                                                            11456
https://dennikn.sk/980405/nie-sme-vhodni-pre-deti-upozornuje-metalova-kaviaren-v-bratislave-po-zlych-skusenostiach/      9060
https://dennikn.sk/977427/vobec-nepotrebujete-predajnu-tiger-rozmyslite-si-ci-tam-nakupujete/                            8590
https://dennikn.sk/976798/jan-figel-schytal-som-to/                                                                      7028
Name: url, dtype: int64

In [429]:
df_dict['events'].loc[df_dict['events']['email'] != '', 'url'].value_counts().head()

    1080302
Name: url, dtype: int64

In [430]:
df_dict['events'].loc[df_dict['events']['url'] != '', 'email'].value_counts().head()

    1528692
Name: email, dtype: int64

In [426]:
df_dict['events'].loc[df_dict['events']['email'] == '', 'mail_job_batch_id'].value_counts().head()

    1529104
Name: mail_job_batch_id, dtype: int64

In [428]:
events_missing_values_share['email'] + events_missing_values_share['url']

0.99984210966020615

In [470]:
print(df_dict['commerce'].columns)
for column in df_dict['commerce'].columns:
    if column not in ['revenue']:
        df_dict['commerce'][column] = df_dict['commerce'][column].astype(str)
        df_dict['commerce'][column] = df_dict['commerce'][column].str.replace('nan', '')
    else:
        df_dict['commerce']['revenue'] = df_dict['commerce']['revenue'].astype(float)
        df_dict['commerce']['revenue'] = df_dict['commerce']['revenue'].fillna(0)
df_dict['commerce'].head()

Index(['article_id', 'author_id', 'currency', 'funnel_id', 'host', 'name',
       'product_ids', 'revenue', 'step', 'time', 'token', 'transaction_id',
       'url', 'user_agent', 'user_id', 'utm_campaign', 'utm_content',
       'utm_medium', 'utm_source'],
      dtype='object')


,article_id,author_id,currency,funnel_id,host,name,product_ids,revenue,step,time,token,transaction_id,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,,,,19,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00000,checkout,1516060958000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,82372,16fd6c84-d6c9-4474-b90c-a548f8ab39ba,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
1,,,,19,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00000,checkout,1516061009000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,94521,,,,
2,,,EUR,19,npress-influxdb-n2.ewhs.bntb.net,commerce,84,4.99000,payment,1516061273000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,7826673105.0,https://dennikn.sk/,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:57.0) G...,56171,1589494e-5a03-4535-a92f-a7c19381f054,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
3,,,EUR,19,npress-influxdb-n2.ewhs.bntb.net,commerce,84,4.99000,purchase,1516061341000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,7826673105.0,https://dennikn.sk/,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:57.0) G...,56171,1589494e-5a03-4535-a92f-a7c19381f054,3a2af47f-c45e-48ec-b2f0-9c4040ebe1b1,inline,remp_campaign
4,,,,19,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00000,checkout,1516061529000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,14524,,,,


In [432]:
commerce_missing_values_share = get_missing_values_share('commerce')
commerce_missing_values_share

article_id       0.02961
author_id        0.02951
currency         0.44585
funnel_id        0.77288
host             1.00000
name             1.00000
product_ids      0.44585
revenue          0.44585
step             1.00000
time             1.00000
token            1.00000
transaction_id   0.44585
url              0.22375
user_agent       0.22722
user_id          1.00000
utm_campaign     0.22732
utm_content      0.15620
utm_medium       0.22729
utm_source       0.22729
dtype: float64

In [471]:
df_dict['commerce']['step'].value_counts(normalize=True)

checkout   0.55415
purchase   0.32373
payment    0.12212
Name: step, dtype: float64

In [472]:
df_dict['commerce']['step'].loc[df_dict['commerce']['product_ids'] != ''].value_counts()

purchase    13525
payment      5102
Name: step, dtype: int64

In [473]:
df_dict['commerce']['step'].loc[df_dict['commerce']['revenue'] != 0].value_counts()

purchase    13525
payment      5102
Name: step, dtype: int64